# E-MOSAIC

引用自[E. R. Q. Fernandes, A. C. P. L. F. de Carvalho and X. Yao, "Ensemble of Classifiers Based on Multiobjective Genetic Sampling for Imbalanced Data," in IEEE Transactions on Knowledge and Data Engineering, vol. 32, no. 6, pp. 1104-1115, 1 June 2020, doi: 10.1109/TKDE.2019.2898861.]


## 数据集的预处理 

In [1]:
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from utils.dataset_utils import get_classes_indexes_counts
import scipy.io as sio  # 从.mat文件中读取数据集
from ucimlrepo import fetch_ucirepo

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print("#########################加载数据集#########################")
'''
id = 12 balance_scale  
'''
# 数据集
uci_dataset = fetch_ucirepo(id=12)
#mat_data = sio.loadmat('../../data/dataset/USPS.mat') 
# 提取变量
features = uci_dataset.data.features  # 特征数据
targets = uci_dataset.data.targets  # 标签lable
# 将数据由dataframe转换成numpy格式

dataset_x = features.to_numpy()
dataset_y = targets.to_numpy()[:, 0]

# dataset_x = mat_data['X']
# dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]

# 显示数据集分布
print("特征数据:", dataset_x.shape)
print("label:", dataset_y.shape)

# One-hot encode target variable 强制将类别转换为0-1序列，0表示不是该类，1表示属于该类
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(dataset_y.reshape(-1, 1))
print("label:", y_onehot.shape)
# 统计每个类别的个数，dataset_y.max()+1是类别的个数
classes, counts = get_classes_indexes_counts(
    np.argmax(y_onehot, axis=1))  #np.argmax(y_onehot, axis=1)找最大值的索引，将0-1序列转化为0,1,2,3......的整数标签
print("每种类别的数量：", counts)

#############################################划分数据集##################################
print("#########################划分数据集#########################")

x_train, x_test, y_train, y_test = train_test_split(dataset_x, y_onehot, test_size=0.3, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# 显示数据集分布
print("特征数据:", x_train.shape)
print("label:", y_train.shape)

# 统计每个类别的个数 np.argmax(y_train, axis=1) Convert one-hot encoded test labels back to single class labels
classes_train, counts_train = get_classes_indexes_counts(np.argmax(y_train, axis=1))
print("训练集每种类别的数量：", counts_train)

classes_test, counts_test = get_classes_indexes_counts(np.argmax(y_test, axis=1))
print("测试集每种类别的数量：", counts_test)

print("#########################平衡数据集#########################")
# 确定每个类别的数量
num_instances = int(counts_train.min() * 0.9)  # 向下取整
print("最小数量:", num_instances)

# 在每个类别中随机的选择该数量的实例的索引
balanced_classes = np.array([])
for indexes in classes_train:
    random_selecte_indices = np.random.choice(indexes, size=num_instances, replace=False)
    balanced_classes = np.hstack((balanced_classes, random_selecte_indices))
balanced_classes = np.sort(balanced_classes).astype(int)

# 得到平衡的数据集
balanced_dataset_x = []
balanced_dataset_y = []
for index in balanced_classes:
    balanced_dataset_x.append(x_train[index])
    balanced_dataset_y.append(y_train[index, :])
balanced_dataset_x = np.array(balanced_dataset_x)
balanced_dataset_y = np.array(balanced_dataset_y).astype(int)

# 显示数据集分布
print("平衡的数据集的特征数据:", balanced_dataset_x.shape)
print("label:", balanced_dataset_y.shape)

# 统计每个类别的个数
classes_balanced_dataset, counts_balanced_dataset = get_classes_indexes_counts(np.argmax(balanced_dataset_y, axis=1))
print("平衡的数据集中每种类别的数量：", counts_balanced_dataset)

#########################加载数据集#########################
特征数据: (625, 4)
label: (625,)
label: (625, 3)
每种类别的数量： [ 49 288 288]
#########################划分数据集#########################
特征数据: (437, 4)
label: (437, 3)
训练集每种类别的数量： [ 31 208 198]
测试集每种类别的数量： [18 80 90]
#########################平衡数据集#########################
最小数量: 27
平衡的数据集的特征数据: (81, 4)
label: (81, 3)
平衡的数据集中每种类别的数量： [27 27 27]


## 评价函数
（G-mean,mAUC两个目标）

In [2]:
from scipy.stats import gmean
from sklearn.metrics import precision_score, roc_auc_score, accuracy_score
from scipy.stats import mode


##########################由个体得到选择的实例子集的索引###########################
def get_indices(individual):
    '''
    :param individual: individual（用实值进行编码）
    :return: 被选择实例的索引
    '''
    individual = np.round(individual)  # 数据范围在0-1之间，转化成int的同时会舍去小数部分，从而将个体映射到0-1编码
    indices = np.where(individual == 1)  # 1代表选择该实例，返回值是tuple，tuple[0]取元组中的第一个元素
    return indices[0]


###########################获取实例子集############################
def get_subset(individual):
    '''
    :param individual: 
    :return: 实例子集
    '''
    indices = get_indices(individual)
    x_sub = balanced_dataset_x[indices, :]
    y_sub = balanced_dataset_y[indices, :]
    return x_sub, y_sub

##########################适应度函数（PPV和PFC，为主要、次要指标）#################################
def fitness_function(individual):
    # 使用训练数据进行预测
    index_pred = individual.mlp.predict(x_test)  # 计算accuracy、PPV
    index_pred_proba = individual.mlp.predict_proba(x_test)  # 计算mAUC
    print(index_pred_proba.shape)
    print(index_pred_proba)

    # Convert one-hot encoded test labels back to single class labels
    y_test_labels = np.argmax(y_test, axis=1)
    print(y_test_labels.shape)
    print(y_test_labels)
    
    y_pred_labels = np.argmax(index_pred, axis=1)
    _, counts = get_classes_indexes_counts(y_test_labels)

    ######################G-mean#########################
    # 计算混淆矩阵
    cm = confusion_matrix(y_test_labels, y_pred_labels)

    # 计算每类召回率（每类正确预测个数 / 该类总数）
    recall_per_class = cm.diagonal() / cm.sum(axis=1)

    # 计算G-Mean
    g_mean = np.prod(recall_per_class) ** (1 / len(recall_per_class))
    geometric_mean = gmean(recall_per_class)
    print("两种G-mean的计算结果：", geometric_mean, g_mean)
    ######################mAUC#######################
    # 计算 ROC AUC（ovo+macro）
    auc_ovo_macro = roc_auc_score(y_test_labels, index_pred_proba, multi_class="ovo", average="macro")
    return round(geometric_mean, 4), round(auc_ovo_macro, 4)


# 集成分类器的投票
def vote_ensembles(save_ensembles):
    y_pred_labels_ensembles = []
    y_test_labels = np.argmax(y_test, axis=1)
    for ensemble in save_ensembles:
        index_pred = ensemble.predict(x_test)  # 计算accuracy、PPV
        # Convert one-hot encoded test labels back to single class labels
        y_pred_labels = np.argmax(index_pred, axis=1)
        y_pred_labels_ensembles.append(y_pred_labels)
    # 按列投票，取每列中出现次数最多的类别作为最终分类结果
    final_pred_result = mode(y_pred_labels_ensembles, axis=0, keepdims=False).mode.flatten()
    cm = confusion_matrix(y_test_labels, final_pred_result)
    # 计算每类召回率（每类正确预测个数 / 该类总数）
    recall_per_class = cm.diagonal() / cm.sum(axis=1)

    # 计算G-Mean
    geometric_mean = gmean(recall_per_class)
    # 计算准确率
    accuracy = accuracy_score(y_test_labels, final_pred_result)
    print(f'Accuracy: {accuracy:.2f}')

    # 打印分类报告
    print("Classification Report:")
    print(classification_report(y_test_labels, final_pred_result))

    # 打印混淆矩阵
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_labels, final_pred_result))
    return geometric_mean


from array import array


# 在种群中找到重复的个体
def find_duplicates(pop, similar=0.85):
    """
    找到重复个体的索引。
    :param arrays: 一个包含 array.array 的列表
    :param threshold: 重复的判断阈值
    :return: 重复对的索引列表
    """
    n = len(pop)
    duplicates = []  # 用于记录重复对的索引

    for i in range(n):
        for j in range(i + 1, n):
            # 当前两组数组
            a = pop[i]
            b = pop[j]

            # 计算1的个数
            ones_a = sum(a)
            ones_b = sum(b)

            # 如果其中一个数组全是0，不可能满足条件
            if ones_a == 0 or ones_b == 0:
                continue

            # 计算交集中的1的数量
            common_ones = sum(x & y for x, y in zip(a, b))

            # 判断是否满足重复的定义
            if (common_ones / ones_a > similar) and (common_ones / ones_b > similar):
                duplicates.append((i, j))

    return duplicates


# 根据索引对，去除种群中重复的个体
def remove_duplicates(pop, duplicates):
    """
    移除重复的个体。
    :param arrays: 一个包含 array.array 的列表
    :param duplicates: 重复对的索引列表
    :return: 去重后的列表
    """
    # 找到所有需要移除的索引
    to_remove = set(j for _, j in duplicates)  # 只保留后出现的索引
    # 构造去重后的列表
    return [pop[i] for i in range(len(pop)) if i not in to_remove]

## NDGA-II

In [3]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from instance_selection.nsga_2.genetic_operator import selNSGA2, mutate_binary_inversion, selTournamentDCD
import warnings

warnings.filterwarnings("ignore")  # 忽略警告
from sklearn.neural_network import MLPClassifier

import array
import random
import matplotlib.pyplot as plt
from deap import base
from deap import creator
from deap import tools

# 最大化评价目标
creator.create("FitnessMaxAndMax", base.Fitness, weights=(1.0, 1.0))
'''
fitness:适应度：Gmean和mAUC
pfc：每个分类器的成对故障信用，用于评估分类器集合的多样性
'''
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMaxAndMax, pfc=None, mlp=None)
toolbox = base.Toolbox()

NDIM = num_instances

# 二进制编码
toolbox.register("attr_binary", random.randint, 0, 1)  # 0-1编码
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_binary, n=num_instances)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)

# 单点交叉
toolbox.register("mate", tools.cxOnePoint)

# 二进制突变
toolbox.register("mutate", mutate_binary_inversion)

# NSGA-II选择
toolbox.register("select", selNSGA2, x_test=X_test_scaled, y_test=y_test)

# 找到种群中重复个体的索引对
toolbox.register("find_duplicates", find_duplicates)

# 去重
toolbox.register("remove_duplicates", remove_duplicates)

# 创建一个MLP模型
init_mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=500, random_state=42)


# 绘制Pareto Front曲线
def plot_front(fronts, gen, title):
    """绘制当前代非支配排序的第一等级前沿"""
    fitnesses = [ind.fitness.values for ind in fronts]
    plt.scatter(*zip(*fitnesses), marker='o', label=f"Generation {gen}")
    plt.title(title)
    plt.xlabel("mAUC")
    plt.ylabel("G-mean")
    plt.legend()
    plt.grid()
    plt.show()
    plt.close()

## 种群的迭代

In [4]:
def main(seed=None):
    random.seed(seed)

    NGEN = 30  # 迭代次数
    POPSIZE = 40  # 种群数量
    CXPB = 1.0  # 交叉因子/交叉率
    MR = 0.2  # 突变因子/突变率
    SIMILAR = 0.85

    ####################################迭代过程的记录#############################
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "fronts", "duplicates"
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)
    ####################################计算初始种群的适应度###########################
    stop_sign = 0
    ensembles = []  # 当前每个个体对应的mlp模型
    save_ensembles = []  # 保存最好的集成模型
    current_ensembles = []  # 存储当前种群对应的集成模型
    g_means_record = []
    #pop_x_sub = []  # 当前每个个体的实例选择的特征数据
    #pop_y_sub = []  # 当前每个个体对应的实例选择的lable
    duplicates = []
    # 对于每个个体都训练得到一个mlp模型
    for i in range(len(pop)):
        mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=500, random_state=42)
        x_sub, y_sub = get_subset(pop[i])
        mlp.fit(x_sub, y_sub)
        ensembles.append(mlp)
        #pop_x_sub.append(x_sub)
        #pop_y_sub.append(y_sub)
        pop[i].mlp = mlp

    # 由mlp模型得到个体的适应度
    for i in range(len(pop)):
        pop[i].fitness.values = toolbox.evaluate(pop[i])

    #################################计算PFC并进行非支配排序#########################################
    # 计算PFC并进行非支配排序 PFC代替拥挤距离
    pop, pareto_fronts = toolbox.select(pop, len(pop))
    # 保存第一个pareto_front中的模型，进行集成
    for ind in pareto_fronts[0]:
        save_ensembles.append(ind.mlp)
    g_mean = vote_ensembles(save_ensembles)
    g_means_record.append(g_mean)
    
    # record = stats.compile(pop)
    # logbook.record(gen=0, evals=len(pop), **record)
    # print(logbook.stream)
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        # 清空当前的集成分类器
        current_ensembles.clear()
        # 选择
        offspring = selTournamentDCD(pop, POPSIZE)
        offspring = [toolbox.clone(ind) for ind in offspring]
        # 交叉
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])
            # 突变
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]
            del offspring[i].fitness.values, offspring[i + 1].fitness.values

        # 计算新的种群适应度 
        ensembles.clear()
        #pop_x_sub.clear()
        #pop_y_sub.clear()
        for i in range(len(offspring)):
            mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=500, random_state=42)
            x_sub, y_sub = get_subset(offspring[i])
            mlp.fit(x_sub, y_sub)
            ensembles.append(mlp)
            #pop_x_sub.append(x_sub)
            #pop_y_sub.append(y_sub)
            offspring[i].mlp = mlp
        for i in range(len(offspring)):
            offspring[i].fitness.values = toolbox.evaluate(offspring[i])

        # 种群的合并
        new_pop = pop + offspring

        # 找到重复个体的索引对
        duplicates = toolbox.find_duplicates(new_pop, SIMILAR)
        # 去除对应重复的个体
        new_pop = toolbox.remove_duplicates(new_pop, duplicates)
        ###########################################再次进行突变################################################
        for ind in range(len(offspring)):
            # 突变
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]
            del offspring[i].fitness.values
        # 计算新的种群适应度 
        ensembles.clear()
        #pop_x_sub.clear()
        #pop_y_sub.clear()
        for i in range(len(offspring)):
            mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=500, random_state=42)
            x_sub, y_sub = get_subset(offspring[i])
            mlp.fit(x_sub, y_sub)
            ensembles.append(mlp)
            #pop_x_sub.append(x_sub)
            #pop_y_sub.append(y_sub)
            offspring[i].mlp = mlp
        for i in range(len(offspring)):
            offspring[i].fitness.values = toolbox.evaluate(offspring[i])

        # 种群的合并
        pop = new_pop + offspring
        ###############################################得到pareto_fronts############################################
        pop, pareto_fronts = toolbox.select(pop, POPSIZE)
        plot_front(pareto_fronts[0], gen, title="Pareto Front (Current Generation)")
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), fronts=len(pareto_fronts[0]), duplicates=len(duplicates), **record)
        print(logbook.stream)
        # 保存第一个等级里的mlp模型进行集成
        for ind in pareto_fronts[0]:
            current_ensembles.append(ind.mlp)
        g_mean = vote_ensembles(current_ensembles)
        if g_mean > g_means_record[-1]:  # 当前的gmean与列表最后一个gmean做对比
            save_ensembles = current_ensembles
            g_means_record.append(g_mean)
            stop_sign = 0
        else:
            stop_sign += 1
        if stop_sign == 10:
            break
    return pop, logbook, save_ensembles


if __name__ == "__main__":
    pop, stats, ensembles = main()

    print("##############################集成分类器的预测结果：################################")
    vote_ensembles(ensembles)

(188, 3)
[[0.05271551 0.29617087 0.84628544]
 [0.01061369 0.99891548 0.00919106]
 [0.03791053 0.9928169  0.0429382 ]
 [0.19088222 0.82682686 0.26024535]
 [0.01341614 0.99919173 0.00601195]
 [0.03585722 0.23952423 0.58703447]
 [0.1867263  0.02827049 0.87640188]
 [0.42764365 0.5593543  0.04913278]
 [0.34865051 0.26087583 0.21273017]
 [0.04206066 0.05768132 0.95956899]
 [0.09812546 0.11568723 0.75212157]
 [0.58201627 0.07342201 0.27101773]
 [0.21387458 0.89950003 0.02960513]
 [0.05498975 0.99031744 0.05650526]
 [0.1644235  0.54385996 0.24264794]
 [0.22118457 0.89498108 0.01991271]
 [0.58883129 0.00689009 0.34221198]
 [0.10804699 0.85598938 0.38336759]
 [0.03421675 0.11228703 0.86017647]
 [0.22006423 0.26080523 0.48981954]
 [0.50515303 0.0260775  0.42174508]
 [0.0158809  0.00723666 0.99677773]
 [0.01629037 0.01231607 0.99655694]
 [0.0072005  0.98935441 0.08110532]
 [0.15996939 0.44676415 0.4591901 ]
 [0.0234404  0.99823294 0.01753327]
 [0.38061617 0.01942039 0.6137655 ]
 [0.28723444 0.6514

ValueError: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes